In [1507]:
"""
------------------------------------------------------------------------------
Libraries
------------------------------------------------------------------------------
"""
from datawrapper import Datawrapper 

import geopandas as gpd
import os
import json
import time
import copy
import math

from requests.exceptions import ReadTimeout

folder_maps = 272196
# https://datawrapper.readthedocs.io/en/latest/user-guide/api.html

In [1508]:
with open('./settings/map_settings.json', 'r') as file:
    map_settings = json.load(file)
with open('./settings/marker_settings.json', 'r') as file:
    marker_settings = json.load(file)
with open('./settings/bezirke_colors.json', 'r') as file:
    bezirke_colors = json.load(file)
with open('./settings/bezike_lookup.json', 'r') as file:
    bezike_lookup = json.load(file)

In [1509]:
# API access
with open("dw_access_token.txt", "r") as file:
    dw_access_token= file.read()
dw = Datawrapper(access_token = dw_access_token)

In [1510]:
# the chart 
# dw.get_chart("EeBGh")

In [1511]:
# def get_bounds_zoom(bounds, width_px, height_px):

#     tile_size = 256
#     min_lon, min_lat, max_lon, max_lat = bounds

#     # Define earth radius and tile size in degrees
#     EARTH_RADIUS = 6378137  # meters
    
#     # Convert latitudes and longitudes to meters using Web Mercator projection
#     def lat_to_meters(lat):
#         return math.log(math.tan((math.pi / 4) + (math.radians(lat) / 2))) * EARTH_RADIUS

#     def lon_to_meters(lon):
#         return math.radians(lon) * EARTH_RADIUS

#     # Width and height of bounds in meters
#     bounds_width_meters = abs(lon_to_meters(max_lon) - lon_to_meters(min_lon))
#     bounds_height_meters = abs(lat_to_meters(max_lat) - lat_to_meters(min_lat))
    
#     # Calculate scale factors for each dimension
#     scale_x = bounds_width_meters / width_px
#     scale_y = bounds_height_meters / height_px
    
#     # Calculate the zoom level using the smaller scale factor
#     scale = min(scale_x, scale_y)
#     zoom = math.log2((tile_size * 256) / scale)
#     zoom = round(zoom, 1)  # Keep one decimal place for precision
#     zoom = zoom - 0.2
#     return max(0, min(zoom, 21))  # Clamp zoom level between 0 and


In [1512]:
def get_bounds_zoom_level(bounds, width_px, height_px, padding = 0):

    min_lon, min_lat, max_lon, max_lat = bounds

    TILE_SIZE = 512
    ZOOM_MAX = 21

    def lat_rad(lat):
        sin_val = math.sin(lat * math.pi / 180)
        rad_x2 = math.log((1 + sin_val) / (1 - sin_val)) / 2
        return max(min(rad_x2, math.pi), -math.pi) / 2

    def zoom(map_px, world_px, fraction):
        # Adjust map dimension by subtracting padding on each side
        adjusted_map_px = map_px - 2 * padding
        return round(math.log(adjusted_map_px / world_px / fraction) / math.log(2), 1)


    # Define the northeast and southwest corners using the new bounds format
    ne = {"lng": max_lon, "lat": max_lat}
    sw = {"lng": min_lon, "lat": min_lat}

    # Calculate the latitude and longitude fractions
    lat_fraction = (lat_rad(ne["lat"]) - lat_rad(sw["lat"])) / math.pi

    lng_diff = ne["lng"] - sw["lng"]
    lng_fraction = (lng_diff + 360 if lng_diff < 0 else lng_diff) / 360

    # Calculate the zoom levels
    lat_zoom = zoom(height_px, TILE_SIZE, lat_fraction)
    lng_zoom = zoom(width_px, TILE_SIZE, lng_fraction)

    return min(lat_zoom, lng_zoom, ZOOM_MAX)

In [1513]:
def get_map_view(geojson_data, width_px, height_px, tile_size=256):
    # Load the GeoJSON data into a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(geojson_data["features"])
    
    # Ensure we have geometries to work with
    if gdf.empty:
        raise ValueError("No valid geometries found in the provided GeoJSON data.")
    
    # Merge all geometries into a single geometry
    unified_geometry = gdf.geometry.unary_union
    
    # Calculate the bounding box
    min_x, min_y, max_x, max_y = unified_geometry.bounds
    
    # Calculate the middle points of each side
    bbox = {
        'top': [(min_x + max_x) / 2, max_y],
        'left': [min_x, (min_y + max_y) / 2],
        'right': [max_x, (min_y + max_y) / 2],
        'bottom': [(min_x + max_x) / 2, min_y]
    }

     # Calculate the center as the midpoint between top and bottom, and left and right
    center = [
        (bbox['left'][0] + bbox['right'][0]) / 2,  # Midpoint of x-coordinates
        (bbox['top'][1] + bbox['bottom'][1]) / 2   # Midpoint of y-coordinates
    ]

    bounds = unified_geometry.bounds
    zoom = get_bounds_zoom_level(bounds, width_px, height_px, 60)


    return {
        'bbox': bbox,
        'center': center,
        'zoom': zoom ,
        'test': bounds
    }


In [1514]:
def export_with_retries(chart_id, chart_name, max_attempts=2):
    attempt = 1
    while attempt <= max_attempts:
        try:
            dw.export_chart(
                chart_id=chart_id,
                scale=2,
                output='png',
                plain=True,
                width=1000,
                border_width=0, 
                filepath=f"./png/{chart_name}.png"
            )
            return f"success (attempt {attempt})! name: {chart_name} id: {chart_id}"
        
        except ReadTimeout:
            print(f"Attempt {attempt} failed due to timeout. Retrying...")
            attempt += 1
            time.sleep(1)  # Adding a short delay between retries (optional)
    
    # If all attempts fail
    return f"fail after {max_attempts} attempts! name: {chart_name} id: {chart_id}"

In [1515]:
def create_map(chart_name,map_view, pr_geojson, pr_imbiss, pr_strassen):
    
    map_settings_pr = copy.deepcopy(map_settings)
    map_settings_pr['visualize']['view']['center'] = map_view['center']
    # map_settings_pr['visualize']['view']['bbox'] = map_view['bbox']
    map_settings_pr['visualize']['view']['zoom'] = map_view['zoom']


    locator_map = dw.create_chart(
        title = chart_name,
        chart_type = "locator-map",
        folder_id = folder_maps,
        metadata = map_settings_pr,
    )
    
    chart_id = locator_map['publicId']

    # marker planungsraum
    bezirk = bezike_lookup[chart_name]
    bezirk_color=bezirke_colors[bezirk]
    marker_pr = copy.deepcopy(marker_settings)
    marker_pr['id'] = 'pr'
    marker_pr['type'] = 'area'
    marker_pr['feature'] = pr_geojson
    marker_pr['properties'] = {
        "fill": bezirk_color,
        "fill-opacity": 0.6,
        "stroke": "#1e3791",
        "stroke-width": 4,
    }

    # marker imbiss
    if pr_imbiss:
        marker_imbiss = copy.deepcopy(marker_settings)
        marker_imbiss['id'] = 'imbiss'
        marker_imbiss['type'] = 'area'
        marker_imbiss['feature'] = pr_imbiss
        marker_imbiss['properties'] = {
            "stroke": "#1e3791",
            "stroke-width": 10,
        }

    # marker strassen
    if pr_strassen:
        marker_strassen = copy.deepcopy(marker_settings)
        marker_strassen['id'] = 'strassen'
        marker_strassen['type'] = 'line'
        marker_strassen['feature'] = pr_strassen
        marker_strassen['properties'] = {
            "stroke":"#e60032",
            "stroke-width":4,
            "stroke-opacity":1,
            "stroke-dasharray":"100000"
        }

    markers = {
        "markers": [
            marker
            for marker in [marker_strassen if "marker_strassen" in locals() else None,
                        marker_imbiss if "marker_imbiss" in locals() else None,
                        marker_pr]
            if marker is not None
        ]
    }

    dw.add_json(
        chart_id = chart_id,
        data = markers
    )

    time.sleep(10)
    
    return export_with_retries(chart_id, chart_name)

In [1516]:
main_folder = "./prognoseraum/"
png_folder = "./png/"

for prognoseraum_name in os.listdir(main_folder):

    if prognoseraum_name == ".DS_Store": #igore .DS_Store file
        continue

    # Check if PNG file already exists in png_folder
    png_path = os.path.join(png_folder, f"{prognoseraum_name}.png")
    if os.path.exists(png_path):
        print(f"Skipping {prognoseraum_name}: PNG already exists.")
        continue

    prognoseraum_path = os.path.join(main_folder, prognoseraum_name)
    
    with open(f"{prognoseraum_path}/{prognoseraum_name}.geojson", 'r') as file:
        pr_geojson = json.load(file)
    
    try:
        with open(f"{prognoseraum_path}/{prognoseraum_name}_imbiss.geojson", 'r') as file:
            pr_imbiss = json.load(file)

    except FileNotFoundError:
        pr_imbiss = False

    try:
        with open(f"{prognoseraum_path}/{prognoseraum_name}_strassen.geojson", 'r') as file:
            pr_strassen = json.load(file)
    
    except FileNotFoundError:
        pr_strassen = False

    map_view = get_map_view(pr_geojson,600,480)
    
    chart_creation_result = create_map(prognoseraum_name,map_view,pr_geojson,pr_imbiss,pr_strassen)

    print(chart_creation_result)


Skipping Zehlendorf_Nord_Wannsee: PNG already exists.
Skipping Gatow___Kladow: PNG already exists.
success (attempt 1)! name: Friedrichshain_West id: Ewidy
Skipping Gropiusstadt: PNG already exists.
success (attempt 1)! name: Schöneberg_Süd id: GrCfC
Skipping Gesundbrunnen: PNG already exists.
Skipping Buch: PNG already exists.
Skipping Haselhorst___Siemensstadt: PNG already exists.
Skipping Neukölln: PNG already exists.
Skipping Grüner_Norden: PNG already exists.
Skipping Britz_Buckow: PNG already exists.
Skipping Südliches_Weißensee: PNG already exists.
Skipping Lichtenberg_Süd: PNG already exists.
Skipping Nördliches_Pankow: PNG already exists.
Skipping Mariendorf: PNG already exists.
Skipping Lankwitz_Lichterfelde_Ost: PNG already exists.
Skipping Marzahn: PNG already exists.
Skipping Südliches_Pankow: PNG already exists.
Skipping Tegel: PNG already exists.
Skipping Kreuzberg_Ost: PNG already exists.
Skipping Schöneberg_Nord: PNG already exists.
Skipping Wedding: PNG already exists